In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
import random

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

random.seed(0)
weight_decay = 0.000005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_fashion_mnist():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(np.pad(X_train, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_train = np.expand_dims(X_train, axis=get_channels_axis())
    X_test = normalize_minus1_1(cast_to_floatx(np.pad(X_test, ((0, 0), (2, 2), (2, 2)), 'constant')))
    X_test = np.expand_dims(X_test, axis=get_channels_axis())
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'fashion-mnist': ('t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag',
                          'ankle-boot'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
def initial_conv(input):
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Convolution2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Convolution2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Convolution2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = 1 if K.image_data_format() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Convolution2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _transformations_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    gpu_to_use = gpu_q.get()
    print(" Training Class ",single_class_ind)
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    transformer = Transformer(8, 8)
    n, k = (2, 8)
    mdl = create_wide_residual_network(x_train.shape[1:], transformer.n_transforms, n, k)
    mdl.compile('adam',
                'categorical_crossentropy',
                ['acc'])

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    print("START the proposed transformations.................. ")
    transformations_inds = np.tile(np.arange(transformer.n_transforms), len(x_train_task))
    x_train_task_transformed = transformer.transform_batch(np.repeat(x_train_task, transformer.n_transforms, axis=0),
                                                           transformations_inds)
    print("END the proposed transformations......................... ")
    print("START the training.................. ")

    mdl.fit(x=x_train_task_transformed, y=to_categorical(transformations_inds),
            batch_size=BATCH_SIZE, epochs= 5 
            )

    print("END the training.................. ")

    #Estimating the parameters
    def calculate_A(index,alpha,beta):
        A = 1
        for i in range(index):
          A = A*( beta[i]/(alpha[i] + beta[i]) ) 
        return A
    def calculate_B(index,alpha,beta):
        B = 1
        for i in range(index):
          B = B*( (beta[i]*(beta[i]+1))/((alpha[i] + beta[i])*(alpha[i] + beta[i]+1)) )
        return B        
    def estimate_param(y):
      N = y.shape[0]
      K = y.shape[1]
      mean = np.clip( y.mean(axis=0), 10**(-7), None)
      covariance = np.clip( np.asarray([np.cov(y[:,j],y[:,j])[0][0] for j in range(K)]), 10**(-7), None)
      alpha = np.zeros(K)
      beta = np.zeros(K)
      A = np.zeros(K)
      B = np.zeros(K)
      alpha[0] = np.clip( (mean[0]*(mean[0]-covariance[0]-mean[0]**2))/covariance[0] , 10**(-7), None)
      beta[0] = np.clip( alpha[0]*(1-mean[0])/mean[0], 10**(-7), None) 
      for j in range(1,K):
        A[j-1] = np.clip( calculate_A(j,alpha,beta), 10**(-7), None)
        B[j-1] = np.clip( calculate_B(j,alpha,beta), 10**(-7), None)
        alpha[j] = np.clip(( (mean[j]**2/A[j-1])*B[j-1] - mean[j]*(covariance[j]+mean[j]**2) ) / ( A[j-1]*(covariance[j]+mean[j]**2) - (mean[j]**2/A[j-1])*B[j-1] ), 0.0001, None)
        beta[j] = np.clip( (alpha[j]*(A[j-1]-mean[j])) / (mean[j]), 10**(-7), None)
      return alpha, beta

    #Normality Score
    def dirichlet_normality_score(alpha,beta,y):
        N = y.shape[0]
        K = y.shape[1]
        y = np.clip(y,10**(-7),1)
        gamma = np.zeros(len(alpha))
        gamma[:-1] = beta[:-1] - alpha[1:] - beta[1:]
        gamma[-1] = beta[-1] -1
        score1 = np.clip(np.asarray([sum((alpha-1)*np.log(y)[j]) for j in range(y.shape[0])]),-10**7,10**7)
        score2 = np.clip(np.asarray([sum([gamma[i]*np.log(np.clip(1-sum(y[j,:i+1]),10**(-7),None)) for i in range(len(alpha-1))]) for j in range(N)]),-10**7,10**7)
        score3 = np.asarray([gamma[-1]*np.log(y[j][K-1]) for j in range(N)])
        return score1 + score2 + score3

    scores = np.zeros((len(x_test),))
    observed_data = x_train_task
    for t_ind in range(transformer.n_transforms):
        observed_dirichlet = mdl.predict(transformer.transform_batch(observed_data, [t_ind] * len(observed_data)),
                                         batch_size=BATCH_SIZE)
        
        y = observed_dirichlet # Our matrix of observations 
        mle_alpha_t,mle_beta_t = estimate_param(y)
        mle_alpha_t= np.clip(np.nan_to_num(mle_alpha_t, copy=True, nan=10**3, posinf=10**10, neginf=-10*10),-10**7,10**7)
        mle_beta_t= np.clip(np.nan_to_num(mle_beta_t, copy=True, nan=0.1, posinf=10**10, neginf=-10**10),-10**7,10**7)
        x_test_p = mdl.predict(transformer.transform_batch(x_test, [t_ind] * len(x_test)),
                               batch_size=BATCH_SIZE)
        x_test_p = np.nan_to_num(x_test_p, copy=True, nan=0, posinf=None, neginf=None)
        scores += dirichlet_normality_score(mle_alpha_t,mle_beta_t, x_test_p)

    scores /= transformer.n_transforms
    labels = y_test.flatten() == single_class_ind
    
    roc_pr_curve_data(scores, labels)
    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    n_runs = 3
    # Transformations
    for i in range(n_runs):
        print('Run number:',i+1)
        processes = [Process(target=_transformations_experiment,
                             args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]

        for p in processes:
            p.start()
        for p in processes:
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 128

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        (load_fashion_mnist, 'fashion-mnist', 10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run number: 1
 Training Class  0
Wide Residual Network-16-8 created.
START the proposed transformations.................. 
END the proposed transformations......................... 
START the training.................. 
Epoch 1/5


204/530 [==========>...................] - ETA: 29:21 - loss: 4.3224 - acc: 0.01 - ETA: 19:49 - loss: 4.3206 - acc: 0.03 - ETA: 18:33 - loss: 4.3181 - acc: 0.03 - ETA: 18:09 - loss: 4.3157 - acc: 0.03 - ETA: 18:09 - loss: 4.3120 - acc: 0.05 - ETA: 18:01 - loss: 4.3084 - acc: 0.06 - ETA: 17:56 - loss: 4.3056 - acc: 0.06 - ETA: 17:45 - loss: 4.3019 - acc: 0.07 - ETA: 17:35 - loss: 4.2991 - acc: 0.07 - ETA: 17:27 - loss: 4.2948 - acc: 0.08 - ETA: 17:20 - loss: 4.2899 - acc: 0.08 - ETA: 17:17 - loss: 4.2848 - acc: 0.09 - ETA: 17:11 - loss: 4.2793 - acc: 0.09 - ETA: 17:07 - loss: 4.2740 - acc: 0.09 - ETA: 17:03 - loss: 4.2682 - acc: 0.10 - ETA: 16:59 - loss: 4.2621 - acc: 0.10 - ETA: 16:56 - loss: 4.2549 - acc: 0.10 - ETA: 16:52 - loss: 4.2481 - acc: 0.11 - ETA: 16:49 - loss: 4.2413 - acc: 0.11 - ETA: 16:47 - loss: 4.2344 - acc: 0.11 - ETA: 16:44 - loss: 4.2267 - acc: 0.11 - ETA: 16:40 - loss: 4.2187 - acc: 0.12 - ETA: 16:37 - loss: 4.2109 - acc: 0.12 - ETA: 16:34 - loss: 4.2032 - acc: 0.12

408/530 [======================>.......] - ETA: 10:24 - loss: 1.9485 - acc: 0.41 - ETA: 10:23 - loss: 1.9428 - acc: 0.41 - ETA: 10:21 - loss: 1.9371 - acc: 0.41 - ETA: 10:19 - loss: 1.9317 - acc: 0.41 - ETA: 10:17 - loss: 1.9267 - acc: 0.41 - ETA: 10:15 - loss: 1.9212 - acc: 0.41 - ETA: 10:13 - loss: 1.9156 - acc: 0.42 - ETA: 10:11 - loss: 1.9104 - acc: 0.42 - ETA: 10:09 - loss: 1.9052 - acc: 0.42 - ETA: 10:07 - loss: 1.9003 - acc: 0.42 - ETA: 10:05 - loss: 1.8952 - acc: 0.42 - ETA: 10:03 - loss: 1.8902 - acc: 0.42 - ETA: 10:01 - loss: 1.8851 - acc: 0.42 - ETA: 9:59 - loss: 1.8804 - acc: 0.4263 - ETA: 9:57 - loss: 1.8755 - acc: 0.427 - ETA: 9:55 - loss: 1.8707 - acc: 0.427 - ETA: 9:53 - loss: 1.8657 - acc: 0.428 - ETA: 9:52 - loss: 1.8611 - acc: 0.429 - ETA: 9:50 - loss: 1.8562 - acc: 0.430 - ETA: 9:48 - loss: 1.8513 - acc: 0.431 - ETA: 9:46 - loss: 1.8464 - acc: 0.432 - ETA: 9:44 - loss: 1.8415 - acc: 0.433 - ETA: 9:42 - loss: 1.8370 - acc: 0.434 - ETA: 9:40 - loss: 1.8323 - acc: 0.43

530/530 [==============================] - ETA: 3:52 - loss: 1.3224 - acc: 0.555 - ETA: 3:50 - loss: 1.3206 - acc: 0.556 - ETA: 3:48 - loss: 1.3188 - acc: 0.556 - ETA: 3:46 - loss: 1.3169 - acc: 0.557 - ETA: 3:44 - loss: 1.3153 - acc: 0.557 - ETA: 3:42 - loss: 1.3136 - acc: 0.558 - ETA: 3:40 - loss: 1.3119 - acc: 0.558 - ETA: 3:38 - loss: 1.3103 - acc: 0.558 - ETA: 3:36 - loss: 1.3086 - acc: 0.559 - ETA: 3:35 - loss: 1.3068 - acc: 0.559 - ETA: 3:33 - loss: 1.3051 - acc: 0.560 - ETA: 3:31 - loss: 1.3034 - acc: 0.560 - ETA: 3:29 - loss: 1.3017 - acc: 0.561 - ETA: 3:27 - loss: 1.2999 - acc: 0.561 - ETA: 3:25 - loss: 1.2982 - acc: 0.562 - ETA: 3:23 - loss: 1.2966 - acc: 0.562 - ETA: 3:21 - loss: 1.2949 - acc: 0.563 - ETA: 3:19 - loss: 1.2932 - acc: 0.563 - ETA: 3:17 - loss: 1.2915 - acc: 0.563 - ETA: 3:15 - loss: 1.2901 - acc: 0.564 - ETA: 3:13 - loss: 1.2883 - acc: 0.564 - ETA: 3:12 - loss: 1.2866 - acc: 0.565 - ETA: 3:10 - loss: 1.2850 - acc: 0.565 - ETA: 3:08 - loss: 1.2834 - acc: 0.566

204/530 [==========>...................] - ETA: 16:47 - loss: 0.6079 - acc: 0.74 - ETA: 16:43 - loss: 0.5767 - acc: 0.74 - ETA: 16:49 - loss: 0.5558 - acc: 0.76 - ETA: 16:45 - loss: 0.5815 - acc: 0.74 - ETA: 16:44 - loss: 0.5674 - acc: 0.75 - ETA: 16:42 - loss: 0.5586 - acc: 0.75 - ETA: 16:42 - loss: 0.5583 - acc: 0.75 - ETA: 16:40 - loss: 0.5698 - acc: 0.74 - ETA: 16:36 - loss: 0.5675 - acc: 0.75 - ETA: 16:33 - loss: 0.5610 - acc: 0.75 - ETA: 16:31 - loss: 0.5553 - acc: 0.76 - ETA: 16:30 - loss: 0.5538 - acc: 0.76 - ETA: 16:28 - loss: 0.5569 - acc: 0.75 - ETA: 16:25 - loss: 0.5501 - acc: 0.76 - ETA: 16:22 - loss: 0.5439 - acc: 0.76 - ETA: 16:21 - loss: 0.5381 - acc: 0.77 - ETA: 16:19 - loss: 0.5382 - acc: 0.77 - ETA: 16:17 - loss: 0.5382 - acc: 0.77 - ETA: 16:16 - loss: 0.5341 - acc: 0.77 - ETA: 16:14 - loss: 0.5290 - acc: 0.77 - ETA: 16:13 - loss: 0.5313 - acc: 0.77 - ETA: 16:11 - loss: 0.5338 - acc: 0.77 - ETA: 16:09 - loss: 0.5354 - acc: 0.77 - ETA: 16:07 - loss: 0.5326 - acc: 0.77

...

408/563 [====================>.........] - ETA: 11:33 - loss: 0.0383 - acc: 0.99 - ETA: 11:31 - loss: 0.0383 - acc: 0.99 - ETA: 11:29 - loss: 0.0383 - acc: 0.99 - ETA: 11:27 - loss: 0.0383 - acc: 0.99 - ETA: 11:25 - loss: 0.0382 - acc: 0.99 - ETA: 11:23 - loss: 0.0382 - acc: 0.99 - ETA: 11:22 - loss: 0.0382 - acc: 0.99 - ETA: 11:20 - loss: 0.0382 - acc: 0.99 - ETA: 11:18 - loss: 0.0382 - acc: 0.99 - ETA: 11:16 - loss: 0.0382 - acc: 0.99 - ETA: 11:14 - loss: 0.0382 - acc: 0.99 - ETA: 11:12 - loss: 0.0382 - acc: 0.99 - ETA: 11:10 - loss: 0.0382 - acc: 0.99 - ETA: 11:08 - loss: 0.0382 - acc: 0.99 - ETA: 11:06 - loss: 0.0383 - acc: 0.99 - ETA: 11:04 - loss: 0.0383 - acc: 0.99 - ETA: 11:02 - loss: 0.0383 - acc: 0.99 - ETA: 11:01 - loss: 0.0383 - acc: 0.99 - ETA: 10:59 - loss: 0.0383 - acc: 0.99 - ETA: 10:57 - loss: 0.0383 - acc: 0.99 - ETA: 10:55 - loss: 0.0383 - acc: 0.99 - ETA: 10:53 - loss: 0.0383 - acc: 0.99 - ETA: 10:51 - loss: 0.0383 - acc: 0.99 - ETA: 10:49 - loss: 0.0383 - acc: 0.99

563/563 [==============================] - ETA: 4:58 - loss: 0.0382 - acc: 0.999 - ETA: 4:56 - loss: 0.0382 - acc: 0.999 - ETA: 4:54 - loss: 0.0382 - acc: 0.999 - ETA: 4:52 - loss: 0.0382 - acc: 0.999 - ETA: 4:50 - loss: 0.0382 - acc: 0.999 - ETA: 4:48 - loss: 0.0382 - acc: 0.999 - ETA: 4:46 - loss: 0.0382 - acc: 0.999 - ETA: 4:44 - loss: 0.0382 - acc: 0.999 - ETA: 4:42 - loss: 0.0383 - acc: 0.999 - ETA: 4:40 - loss: 0.0383 - acc: 0.999 - ETA: 4:38 - loss: 0.0382 - acc: 0.999 - ETA: 4:36 - loss: 0.0383 - acc: 0.999 - ETA: 4:34 - loss: 0.0383 - acc: 0.999 - ETA: 4:33 - loss: 0.0383 - acc: 0.999 - ETA: 4:31 - loss: 0.0383 - acc: 0.999 - ETA: 4:29 - loss: 0.0383 - acc: 0.999 - ETA: 4:27 - loss: 0.0383 - acc: 0.999 - ETA: 4:25 - loss: 0.0383 - acc: 0.999 - ETA: 4:23 - loss: 0.0383 - acc: 0.999 - ETA: 4:21 - loss: 0.0383 - acc: 0.999 - ETA: 4:19 - loss: 0.0383 - acc: 0.999 - ETA: 4:17 - loss: 0.0383 - acc: 0.999 - ETA: 4:15 - loss: 0.0383 - acc: 0.999 - ETA: 4:13 - loss: 0.0383 - acc: 0.999

204/563 [=========>....................] - ETA: 17:59 - loss: 0.0447 - acc: 1.00 - ETA: 18:09 - loss: 0.0431 - acc: 1.00 - ETA: 18:08 - loss: 0.0519 - acc: 0.99 - ETA: 18:07 - loss: 0.0482 - acc: 0.99 - ETA: 18:01 - loss: 0.0462 - acc: 0.99 - ETA: 17:57 - loss: 0.0453 - acc: 0.99 - ETA: 17:55 - loss: 0.0445 - acc: 0.99 - ETA: 17:55 - loss: 0.0442 - acc: 0.99 - ETA: 17:53 - loss: 0.0436 - acc: 0.99 - ETA: 17:51 - loss: 0.0433 - acc: 0.99 - ETA: 17:49 - loss: 0.0434 - acc: 0.99 - ETA: 17:45 - loss: 0.0429 - acc: 0.99 - ETA: 17:44 - loss: 0.0425 - acc: 0.99 - ETA: 17:41 - loss: 0.0422 - acc: 0.99 - ETA: 17:39 - loss: 0.0423 - acc: 0.99 - ETA: 17:36 - loss: 0.0422 - acc: 0.99 - ETA: 17:36 - loss: 0.0429 - acc: 0.99 - ETA: 17:33 - loss: 0.0425 - acc: 0.99 - ETA: 17:31 - loss: 0.0423 - acc: 0.99 - ETA: 17:30 - loss: 0.0421 - acc: 0.99 - ETA: 17:29 - loss: 0.0424 - acc: 0.99 - ETA: 17:26 - loss: 0.0424 - acc: 0.99 - ETA: 17:24 - loss: 0.0426 - acc: 0.99 - ETA: 17:22 - loss: 0.0428 - acc: 0.99

408/563 [====================>.........] - ETA: 11:32 - loss: 0.0438 - acc: 0.99 - ETA: 11:30 - loss: 0.0439 - acc: 0.99 - ETA: 11:28 - loss: 0.0439 - acc: 0.99 - ETA: 11:26 - loss: 0.0445 - acc: 0.99 - ETA: 11:24 - loss: 0.0445 - acc: 0.99 - ETA: 11:22 - loss: 0.0445 - acc: 0.99 - ETA: 11:20 - loss: 0.0444 - acc: 0.99 - ETA: 11:18 - loss: 0.0447 - acc: 0.99 - ETA: 11:16 - loss: 0.0447 - acc: 0.99 - ETA: 11:14 - loss: 0.0448 - acc: 0.99 - ETA: 11:12 - loss: 0.0451 - acc: 0.99 - ETA: 11:10 - loss: 0.0452 - acc: 0.99 - ETA: 11:08 - loss: 0.0453 - acc: 0.99 - ETA: 11:06 - loss: 0.0456 - acc: 0.99 - ETA: 11:04 - loss: 0.0457 - acc: 0.99 - ETA: 11:02 - loss: 0.0457 - acc: 0.99 - ETA: 11:00 - loss: 0.0458 - acc: 0.99 - ETA: 10:58 - loss: 0.0458 - acc: 0.99 - ETA: 10:57 - loss: 0.0460 - acc: 0.99 - ETA: 10:55 - loss: 0.0460 - acc: 0.99 - ETA: 10:53 - loss: 0.0461 - acc: 0.99 - ETA: 10:51 - loss: 0.0461 - acc: 0.99 - ETA: 10:49 - loss: 0.0462 - acc: 0.99 - ETA: 10:47 - loss: 0.0462 - acc: 0.99

563/563 [==============================] - ETA: 4:57 - loss: 0.0461 - acc: 0.997 - ETA: 4:55 - loss: 0.0460 - acc: 0.997 - ETA: 4:53 - loss: 0.0460 - acc: 0.997 - ETA: 4:52 - loss: 0.0460 - acc: 0.997 - ETA: 4:50 - loss: 0.0460 - acc: 0.997 - ETA: 4:48 - loss: 0.0460 - acc: 0.997 - ETA: 4:46 - loss: 0.0461 - acc: 0.997 - ETA: 4:44 - loss: 0.0460 - acc: 0.997 - ETA: 4:42 - loss: 0.0460 - acc: 0.997 - ETA: 4:40 - loss: 0.0460 - acc: 0.997 - ETA: 4:38 - loss: 0.0460 - acc: 0.997 - ETA: 4:36 - loss: 0.0460 - acc: 0.997 - ETA: 4:34 - loss: 0.0459 - acc: 0.997 - ETA: 4:32 - loss: 0.0460 - acc: 0.997 - ETA: 4:30 - loss: 0.0460 - acc: 0.997 - ETA: 4:28 - loss: 0.0459 - acc: 0.997 - ETA: 4:26 - loss: 0.0459 - acc: 0.997 - ETA: 4:24 - loss: 0.0459 - acc: 0.997 - ETA: 4:23 - loss: 0.0459 - acc: 0.997 - ETA: 4:21 - loss: 0.0459 - acc: 0.997 - ETA: 4:19 - loss: 0.0459 - acc: 0.997 - ETA: 4:17 - loss: 0.0458 - acc: 0.997 - ETA: 4:15 - loss: 0.0458 - acc: 0.997 - ETA: 4:13 - loss: 0.0458 - acc: 0.997